## Exploring the Dataset 

In [3]:
PLOTS = False
trade_plots = False 

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
x_train_path = "D:\Data-cfm\X_train.csv"
y_train_path = "D:\Desktop\Coding-Projects\Prediction-Challenge\Data\y_train.csv"

In [5]:
# x_train = pd.read
# y_train = pd.read_csv(test_path)
# y_train.head()

Data = 504 days × 24 stocks × 20 observations/day × 100 events/observation  

Here is a description of each column in the dataset. <br>

| Column | Description |
| ------ | ------------ |
| **Obs_id** | which observation are we taking into account <br>-> for that observation we will keep track of the next 100 operations in the book orders |
| **Venue_id** | for a given stock, exchanges can happen across many venues :  this id tracks which venue we consider <br> ==> it could be of importance (some stocks are typically traded across many venues ?) |
| **order_id** | for a given observation sequence, each operation is related to an order. An order can be added, updated, deleted. <br>The order_id allows to track the lifecycle of individual orders within a sequence.   |
| **action** |  A (adding an order to the book) , D (Deleting an order from the book), U = updating an action from the book |
| **side** | B (bids, values to buy the action) , A (Ask, values to sell the action) 
| **Price** | - price : price of the order that was affected. *This best_bid_price , at the time of the first event, is substracted from all price reated columns (price, bid, ask  ) |
| **bid , ask** |- bid , ask == best bid (highest bid) /best ask (lowest ask)   |
| **bid_size, ask_size** |  volume of orders at the best bid, respectively ask, price  , on the *aggregated book* <br> => this too could be a valuable information, perhaps some stocks are encoutering more volume than others.  |
|**flux** | the change in volume at a specific price level in the order book due to a particular event |
|**Trade**|A boolean true or false to indicate whether a deletion or update event was due to a trade or due to a cancellation. <br> Most Deletions and updates actually dont occur from Trades

### Example: For a given Observation

| `order_id` | `action` | `price` | `side` | **Description**                                          |
|------------|----------|---------|-------|----------------------------------------------------------|
| 0          | A        | 100.5   | B     | A new order (ID 0) is added at 100.5 on the bid side.    |
| 1          | A        | 101.0   | A     | A new order (ID 1) is added at 101.0 on the ask side.    |
| 0          | U        | 100.5   | B     | The order with ID 0 is updated (e.g., quantity changed). |
| 1          | D        | 101.0   | A     | The order with ID 1 is deleted (removed from the book).  |


# Explore trade info intuition 

In [6]:
if trade_plots:
        
    # Filter actions that are either 'D' or 'U'
    du_actions = df[(df['action'].isin(['D', 'U']))]

    # Count actions where 'trade' is True among 'D' or 'U'
    du_trades = du_actions[du_actions['trade'] == True]

    # Calculate the percentage
    percentage = (len(du_trades) / len(du_actions)) * 100

    # Display the result
    print(f"Percentage of 'D' or 'U' actions coming from trades: {percentage:.2f}%")

    # Merge the main DataFrame (df) with y_train using obs_id
    df = df.merge(y_train, on='obs_id', how='left')  # Assuming y_train has columns ['obs_id', 'stock']

    # Define a function to calculate the percentage for each observation
    def calculate_percentage(sub_df):
        is_du = sub_df['action'].isin(['D', 'U'])
        is_du_trade = is_du & (sub_df['trade'] == True)
        return (is_du_trade.sum() / is_du.sum()) * 100 if is_du.sum() > 0 else 0

    # Group by obs_id and calculate percentage
    df_obs = df.groupby('obs_id').apply(calculate_percentage).reset_index(name='percentage')

    df_obs = df_obs.merge(y_train, on='obs_id', how='left')

    # Group by stock and calculate statistics
    stock_stats = df_obs.groupby('eqt_code_cat')['percentage'].agg(['mean', 'std', 'min', 'max'])

    plt.figure(figsize=(12, 8))
    sns.boxplot(x='eqt_code_cat', y='percentage', data=df_obs)
    plt.title('Distribution of Percentages by Stock')
    plt.xlabel('Stock')
    plt.ylabel('Percentage of D/U Actions from Trades')
    plt.xticks(rotation=90)
    plt.show()


        
    # Cap percentages at 6%
    df_obs_capped = df_obs[df_obs['percentage'] <= 6]

    # Create subplots: One histogram per stock
    stocks = df_obs_capped['eqt_code_cat'].unique()  # Get unique stocks
    num_stocks = len(stocks)

    # Define the number of rows and columns for subplots
    ncols = 4  # Number of columns
    nrows = (num_stocks + ncols - 1) // ncols  # Calculate rows based on number of stocks

    # Create the figure
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows * 3))
    axes = axes.flatten()  # Flatten axes for easier indexing

    # Plot each stock
    for i, stock in enumerate(stocks):
        # Filter the data for the current stock
        stock_data = df_obs_capped[df_obs_capped['eqt_code_cat'] == stock]
        
        # Plot the histogram for the stock
        sns.histplot(
            stock_data,
            x='percentage',
            bins=30,
            ax=axes[i],
            element='step',
            stat='percent'  # Show percentages instead of counts
        )
        axes[i].set_title(f'Stock {stock}', fontsize=12)
        axes[i].set_xlim(0, 6)  # Cap the percentage at 6
        axes[i].set_ylim(0, 100)  # Cap the y-axis at 100%
        axes[i].set_xlabel('Percentage of D/U Actions', fontsize=10)
        axes[i].set_ylabel('Percentage (%)', fontsize=10)

    # Remove unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()



        



There seems to be three modes : 0% of trades , 2% of trades, 4% of trades 

# Some ideas after the initial exploration

For a given observation, what can help determine the stock ?    
we could use visualisation (for a given stock : average volatility observed , average number of increase of orders, average number of decrease of oders etc simple metrics as such)  
  
To go more in depth : we must use embeddings of our data, think of interesting traits, use correlations, try and reduce the dimensionality.  
--> ideas seem endless we could train an embedding matrix to predict the venue idk 

# Reproducing the Benchmark

The benchmark for the challenge is the following architecture :  

Preprocess:  
converting each event into a 30-dimensionnal vector.  
group each 100-event-observations into a single "observation" vector, size 100x30
  
Architecture:  
bidirectionnal GRU network, with 64 hidden units.  Producing a single 128 dimensional vector per "observation vector" .  
Many to one architecture :converts the "observation vector" (of 100 individual events) into a single embedding of size 124.  
Then two dense layers 124 -> 64 with SeLU activation, 64 -> 24 with softmax activation  
  
Training :  
Cross entropy Loss  
batch size : 1000 "obervation vectors"  (dim : 1000x100x30)  
optimizer : Base ADAM with lr = 10e-3

In [7]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense, Activation, Embedding, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector

## Pre processing of the data 


Exact pre process structure isnt described, so I will do what sounds relevant. 

Here is a description of each column in the dataset. <br>

| Column | Description |
| ------ | ------------ |
| **Obs_id** | which observation are we taking into account <br>-> for that observation we will keep track of the next 100 operations in the book orders |
| **Venue_id** | for a given stock, exchanges can happen across many venues :  this id tracks which venue we consider <br> ==> it could be of importance (some stocks are typically traded across many venues ?) |
| **order_id** | for a given observation sequence, each operation is related to an order. An order can be added, updated, deleted. <br>The order_id allows to track the lifecycle of individual orders within a sequence.   |
| **action** |  A (adding an order to the book) , D (Deleting an order from the book), U = updating an action from the book |
| **side** | B (bids, values to buy the action) , A (Ask, values to sell the action) 
| **Price** | - price : price of the order that was affected. *This best_bid_price , at the time of the first event, is substracted from all price reated columns (price, bid, ask  ) |
| **bid , ask** |- bid , ask == best bid (highest bid) /best ask (lowest ask)   |
| **bid_size, ask_size** |  volume of orders at the best bid, respectively ask, price  , on the *aggregated book* <br> => this too could be a valuable information, perhaps some stocks are encoutering more volume than others.  |
|**flux** | the change in volume at a specific price level in the order book due to a particular event |
|**Trade**|A boolean true or false to indicate whether a deletion or update event was due to a trade or due to a cancellation. <br> Most Deletions and updates actually dont occur from Trades

In [8]:
#venue => one hot encode it 
#action => one hot encode it 
#side : => one hot encode it 
#price,bid,ask,bid_size,ask_size,flux : no transfo
#trade : one hot encode it 

#Justifications ? => none, just exploring 


## Preprocess data

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import h5py  # For saving large arrays in memory-efficient HDF5 format
from tqdm import tqdm 

In [10]:
def encode_df(df_to_encode):
    categorical_columns = ['venue','action','side','trade']
    df_pandas_encoded = pd.get_dummies(df_to_encode,columns=categorical_columns,drop_first=False,dtype=int)

    return df_pandas_encoded

In [11]:
def transform_df(df):
    #we want to drop obs_id, order_id 
    df = df.drop(['order_id','obs_id'],axis=1) #dropping obs id because they did so in the benchmark
    return df

In [12]:
# df_test = df.head(int(10e3))
# df_test = encode_df(df_test)
# df_test = transform_df(df_test)
# df_test
# df_test

In [13]:
def create_lstm_data(data, k):
    '''
    input:
        data - the pandas object of (n_observations x 100 , p) shape, where n is the number of rows,
               p is the number of predictors
        k    - the length of the sequences, namely, the number of previous rows 
               (including current) we want to use to predict the target.
    output:
        X_data - the predictors numpy matrix of (n-k, k, p) shape
    '''


    # initialize zero matrix of (n-k, k, p) shape to store the n-k number
    # of sequences of k-length and zero array of (n-k, 1) to store targets
    X_data = np.zeros((data.shape[0]//k, k, data.shape[1]))
    
    # run loop to slice k-number of previous rows as 1 sequence to predict
    # 1 target and save them to X_data matrix and y_data list
    for i in range(data.shape[0]//k):
        cur_sequence = data.iloc[k*i: k*(i+1), :]
                
        X_data[i,:,:] = cur_sequence
    
    return X_data

In [14]:
y_train = np.asarray(pd.read_csv(y_train_path).drop('obs_id',axis=1))

In [15]:
#installing a library to handle out-of-memeory packages https://stackoverflow.com/questions/30376581/save-numpy-array-in-append-mode/64403144#64403144

In [ ]:
# pip install npy-append-array

Note: you may need to restart the kernel to use updated packages.


In [17]:
from npy_append_array import NpyAppendArray

In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def process_data_chunked(x_train_path, y_train_path, output_prefix, chunk_size=10_000, seq_len=100):
    """
    Process data chunk by chunk and save the results incrementally.
    
    Args:
    - x_train_path: Path to the X_train CSV file.
    - y_train_path: Path to the y_train CSV file.
    - output_prefix: Prefix for the output files.
    - chunk_size: Number of rows to process in each chunk.
    - seq_len: Length of each sequence for LSTM.
    """
    # Read y_train (the target file) entirely as it's small and doesn't need chunking
    y_train_full = pd.read_csv(y_train_path).drop('obs_id', axis=1)

    # Use tqdm to show progress
    total_rows = sum(1 for _ in open(x_train_path)) - 1  # Get total rows excluding header
    num_chunks = (total_rows + chunk_size - 1) // chunk_size  # Calculate total chunks

    X_train_npy_name = f"{output_prefix}_X_train.npy"
    y_train_npy_name = f"{output_prefix}_y_train.npy"


    # Process the X_train file in chunks
    with NpyAppendArray(X_train_npy_name, delete_if_exists=True) as npaa:
        for i, chunk in enumerate(tqdm(pd.read_csv(x_train_path, chunksize=chunk_size), desc="Processing Chunks", total=num_chunks)):
            # Apply the transformation functions
            chunk = transform_df(encode_df(chunk))

            # Create LSTM-compatible data for this chunk
            X_data = create_lstm_data(chunk, seq_len)

            npaa.append(X_data)

            # Clear memory for the current chunk
            del X_data, chunk


    np.save(y_train_npy_name,y_train_full)
    print(f"Processing completed. X_train and y_train saved as {output_prefix}_X_train.npy and {output_prefix}_y_train.npy.")


In [19]:
# process_data_chunked(x_train_path, y_train_path, '27-11', chunk_size=10_000, seq_len=100)


Processing Chunks: 100%|██████████| 1608/1608 [01:52<00:00, 14.32it/s]

Processing completed. X_train and y_train saved as 27-11_X_train.npy and 27-11_y_train.npy.


In [20]:
# file_path = r"D:\Desktop\Coding-Projects\Prediction-Challenge\Notebooks\27-11_X_train.npy"

# # Load the .npy file
# data = np.load(file_path)

# # Display the data (e.g., shape, a sample of the contents)
# print("Data Shape:", data.shape)

In [21]:
# y_path = r"D:\Desktop\Coding-Projects\Prediction-Challenge\Notebooks\27-11_y_train.npy"
# # Load the .npy file
# y = np.load(y_path)

# # Display the data (e.g., shape, a sample of the contents)
# print("Data Shape:", y.shape)


## Data generator

In [22]:
160800/10050

16.0

In [23]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs,x_path_npy,y_path_npy, batch_size=10050, dim=(100,19),
                 n_classes=24, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.x_path = x_path_npy
        self.y_path = y_path_npy

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size))

        # Generate data
        X_full = np.load(self.x_path, mmap_mode="r")
        y_full = np.load(self.y_path,mmap_mode="r")

        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = X_full[ID]

            # Store class
            y[i] = y_full[ID].astype(int)[0]
        
        del X_full
        del y_full
        

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [24]:
# Parameters
params = {'dim': (100,19),
          'batch_size': 10050,
          'n_classes': 24,
          'shuffle': True}

# Datasets
full_ids = np.arange(160800)

# Shuffle the IDs to ensure randomness
np.random.shuffle(full_ids)

# Compute the split index for 80/20
split_index = int(len(full_ids) * 13/16) #13/16 test, 3/16 val

# Split the IDs
train_ids = full_ids[:split_index]
val_ids = full_ids[split_index:]

x_path_npy = r'D:\Desktop\Coding-Projects\Prediction-Challenge\Notebooks\27-11_X_train.npy'
y_path_npy = r'D:\Desktop\Coding-Projects\Prediction-Challenge\Notebooks\27-11_y_train.npy'

# Generators
training_generator = DataGenerator(train_ids,x_path_npy,y_path_npy, **params)
val_generator = DataGenerator(val_ids,x_path_npy,y_path_npy, **params)


## Model

Now we have the data in a satisfactory format.  
each row of our X_train is made of 100 event, each of these events is represented in a 18 dimension space.  
and for each row of our train set, we have a single target value : in y  
Let's now create a similar architecture as the benchmark  

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional,LSTM

In [27]:
# Input for fixed-length (length = 100) sequences of event observation (dimension = 19)
inputs = keras.Input(shape=(100,19))

# Add 2 bidirectional LSTMs
x = Bidirectional(LSTM(64))(inputs)
x = Dense(64)(x)
# Add a classifier
outputs = Dense(24, activation="softmax")(x)
model_2 = keras.Model(inputs, outputs)
model_2.summary()

model_2.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-3), loss="categorical_crossentropy", metrics=["accuracy"]) #need sparsecatcrossentropy cause integer labels

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 19)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              43008     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 24)                1560      
                                                                 
Total params: 52,824
Trainable params: 52,824
Non-trainable params: 0
_________________________________________________________________


### Callbacks

In [28]:
# Import necessary libraries for callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

# Define callbacks to enhance and monitor the training process
callbacks = [
    # 1. ModelCheckpoint:
    # Saves the model to a file ('best_model.h5') whenever the validation loss ('val_loss') improves.
    # Ensures that only the best version of the model (with the lowest validation loss) is saved.
    ModelCheckpoint(
        filepath='best_model.h5',   # Filepath to save the model
        monitor='val_loss',        # Metric to monitor
        save_best_only=True,       # Save only the best model
        mode='min'                 # Minimize the 'val_loss'
    ),
    
    # 2. EarlyStopping:
    # Stops training if the validation loss does not improve for 'patience' epochs (5 in this case).
    # Prevents overfitting and saves time by stopping early when progress stalls.
    EarlyStopping(
        monitor='val_loss',        # Metric to monitor
        patience=5,                # Number of epochs to wait without improvement
        mode='min',                # Minimize the 'val_loss'
        restore_best_weights=True  # Restore the model weights from the best epoch
    ),
    
    # 3. ReduceLROnPlateau:
    # Reduces the learning rate when the validation loss plateaus (does not improve for 3 epochs here).
    # Helps the model converge better by lowering the learning rate when progress slows down.
    ReduceLROnPlateau(
        monitor='val_loss',        # Metric to monitor
        factor=0.2,                # Factor by which to reduce the learning rate
        patience=3,                # Number of epochs to wait before reducing the learning rate
        min_lr=1e-6                # Minimum learning rate to avoid reducing it too much
    ),
    
    # 4. TensorBoard:
    # Logs training metrics, such as loss and accuracy, for visualization using TensorBoard.
    # Also logs histograms and the computational graph of the model.
    TensorBoard(
        log_dir='./logs',          # Directory to save TensorBoard logs
        histogram_freq=1,          # Log histograms of weights after every epoch
        write_graph=True,          # Save the computation graph
        write_images=True          # Save visualizations of weights and biases
    ),
    
    # 5. CSVLogger:
    # Logs training and validation metrics to a CSV file ('training_log.csv').
    # Useful for tracking metrics over time and for external analysis.
    CSVLogger(
        filename='training_log.csv',  # Path to save the log file
        append=True                  # Append to existing file if it exists
    )
]

In [29]:
# Train model on dataset
model_2.fit(
    x=training_generator,
    validation_data=val_generator,
    epochs=2,  # Specify the number of epochs as needed
    callbacks = callbacks
)   

Epoch 1/2
13/13 [==============================] - 317s 21s/step - loss: 3.1583 - accuracy: 0.0604 - val_loss: 3.0107 - val_accuracy: 0.0774 - lr: 0.0030
Epoch 2/2
13/13 [==============================] - 450s 34s/step - loss: 2.9522 - accuracy: 0.0840 - val_loss: 2.8996 - val_accuracy: 0.0895 - lr: 0.0030


## Prediction with our trained model 

In [47]:
from random import  randint
sequence = [randint(0,20000) for j in range(100)]

#loading a sequence 

for i in sequence:
        
    A = np.load(x_path_npy,mmap_mode='r')[i] #this is a 100x19 sequence
    A = np.expand_dims(A, axis=0)  # Adding batch dimension
    A.shape
    label = np.load(y_path_npy,mmap_mode='r')[i]

    predicted = model_2.predict(A)

    top_3_indices = predicted[0].argsort()[-3:][::-1]

    # Get the top 3 values
    top_3_values = predicted[0][top_3_indices]

    print("Top 3 predicted labels :", top_3_indices)
    print("with following Top 3 values:", top_3_values)

    #print(f"Label predicted", predicted[0].argmax())
    print(f"True Label",label[0])

1/1 [==============================] - 0s 44ms/step
Top 3 predicted labels : [17  0  9]
with following Top 3 values: [0.12209046 0.09839866 0.09341515]
True Label 9
1/1 [==============================] - 0s 78ms/step
Top 3 predicted labels : [19 23  6]
with following Top 3 values: [0.12720132 0.12183362 0.09871302]
True Label 6
1/1 [==============================] - 0s 92ms/step
Top 3 predicted labels : [ 0 10  4]
with following Top 3 values: [0.0731776 0.0698133 0.0617378]
True Label 21
1/1 [==============================] - 0s 96ms/step
Top 3 predicted labels : [ 3 11  5]
with following Top 3 values: [0.06901506 0.06339215 0.06177472]
True Label 15
1/1 [==============================] - 0s 92ms/step
Top 3 predicted labels : [ 2 23  7]
with following Top 3 values: [0.11479475 0.10407948 0.08718833]
True Label 8
1/1 [==============================] - 0s 86ms/step
Top 3 predicted labels : [ 0 17 22]
with following Top 3 values: [0.14511058 0.13858934 0.12294507]
True Label 9
1/1 [======

In [31]:
# classes = np.load(y_path_npy)
# pd.DataFrame(classes).describe()
# #description of labels classes 